# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

W. Brandner  ->  W. Brandner  |  ['W. Brandner']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
T. Henning  ->  T. Henning  |  ['T. Henning']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']


R. Burn  ->  R. Burn  |  ['R. Burn']
T. Henning  ->  T. Henning  |  ['T. Henning']
M. Kürster  ->  M. Kürster  |  ['M. Kürster']
L. Acuña  ->  L. Acuña  |  ['L. Acuña']
Arxiv has 59 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.02908


extracting tarball to tmp_2504.02908... done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:488: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


✔ → 0:header
  ↳ 7371:\section{Introduction}
✔ → 7371:\section{Introduction}
  ↳ 12370:\section{Observations}
✔ → 12370:\section{Observations}
  ↳ 13252:\section{Yukawa correction to Newtonian force}


...

Found 102 bibliographic references in tmp_2504.03364/aamaintex.bbl.
Issues with the citations
syntax error in line 578: '=' expected
Retrieving document from  https://arxiv.org/e-print/2504.03572


extracting tarball to tmp_2504.03572...

 done.


L. Acuña  ->  L. Acuña  |  ['L. Acuña']


Found 88 bibliographic references in tmp_2504.03572/Gl410_SPIRou_SOPHIE.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.02908-b31b1b.svg)](https://arxiv.org/abs/2504.02908) | **On the presence of a fifth force at the Galactic Center**  |
|| G. Collaboration, et al. -- incl., <mark>W. Brandner</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>S. Scheithauer</mark> |
|*Appeared on*| *2025-04-07*|
|*Comments*| *Submitted to A&A*|
|**Abstract**|            Aims: The presence of a Yukawa-like correction to Newtonian gravity is investigated at the Galactic Center, leading to a new upper limit for the intensity of such a correction. Methods: We perform a Markov Chain Monte Carlo analysis using the astrometric and spectroscopic data of star S$2$ collected at the Very Large Telescope by GRAVITY, NACO and SINFONI instruments, covering the period from $1992$ to $2022$. Results: The precision of the GRAVITY instrument allows us to derive the most stringent upper limit at the Galactic Center for the intensity of the Yukawa contribution ($\propto \, \alpha e^{- \lambda r}$) to be $|\alpha| < 0.003$ for a scale length $\lambda = 3 \cdot 10^{13}\, \rm m\, (\sim 200 \, \rm AU)$. This improves by roughly one order of magnitude all estimates obtained in previous works.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.03364-b31b1b.svg)](https://arxiv.org/abs/2504.03364) | **The CARMENES search for exoplanets around M dwarfs. Occurrence rates of Earth-like planets around very low-mass stars**  |
|| A. Kaminski, et al. -- incl., <mark>R. Burn</mark>, <mark>T. Henning</mark>, <mark>M. Kürster</mark> |
|*Appeared on*| *2025-04-07*|
|*Comments*| *Number of pages: 25; number of figures: 23 (12 in main + 11 in appendix). Accepted in A&A; DOI (pending): https://doi.org/10.1051/0004-6361/202453381 (Volume 696, Article number A101)*|
|**Abstract**|            Aims: Previous estimates of planet occurrence rates in the CARMENES survey indicated increased numbers of planets on short orbits for M dwarfs with masses below 0.34\,M$_\odot$. Here we focused on the lowest-mass stars in the survey, comprising 15 inactive targets with masses under 0.16\,M$_\odot$. Methods: To correct for detection biases, we determined detection sensitivity maps for individual targets and the entire sample. Using Monte Carlo simulations, we estimated planet occurrence rates for orbital periods of 1\,d to 100\,d and minimum masses from 0.5\,M$_\oplus$ to 10\,M$_\oplus$. Results: The radial velocity (RV) data from CARMENES reveal four new planets around three stars in our sample, namely G~268--110\,b, G~261--6\,b, and G~192--15\,b and c. All three b planets have minimum masses of 1.03--1.52\,M$_\oplus$ and orbital periods of 1.43--5.45\,d, while G~192--15\,c is a 14.3\,M$_\oplus$ planet on a wide, eccentric orbit with $P \approx 1218$\,d and $e \approx 0.68$. Our occurrence rates suggest considerable dependencies with respect to stellar masses. For planets below 3\,M$_\oplus$ we found rates consistent with one planet per star across all investigated periods, but the rates decrease almost by an order of magnitude for larger planet masses up to 10\,M$_\oplus$. Compared to previous studies, low-mass stars tend to harbor more planets with $P <10$\,d. We also demonstrate that synthetic planet populations based on the standard core accretion scenario predict slightly more massive planets on wider orbits than observed. Conclusions: Our findings confirm that planet occurrence rates vary with stellar masses even among M dwarfs, as we found more planets with lower masses and on shorter orbits in our subsample of very low-mass stars compared to more massive M dwarfs. Therefore, we emphasize the need for additional differentiation in future studies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.03572-b31b1b.svg)](https://arxiv.org/abs/2504.03572) | **Characterizing planetary systems with SPIRou: Detection of a sub-Neptune in a 6 day period orbit around the M dwarf Gl 410**  |
|| A. Carmona, et al. -- incl., <mark>L. Acuña</mark> |
|*Appeared on*| *2025-04-07*|
|*Comments*| *Accepted by A&A, 4 April 2025; 35 pages*|
|**Abstract**|            The search of exoplanets around nearby M dwarfs is a crucial milestone to perform the census of planetary systems in the vicinity of our Solar System. Since 2018 our team is carrying a radial-velocity blind search program for planets around nearby M dwarfs with the near-IR spectro-polarimeter and velocimeter SPIRou at the CFHT and the optical velocimeter SOPHIE at the OHP in France. Here we present our results on Gl 410, a 0.55 Msun 480+-150 Myr old active M dwarf distant 12 pc. We use the line-by-line (LBL) technique to measure the RVs with SPIRou and the template matching method with SOPHIE. Three different methods, two based in principal component analysis (PCA), are used to clean the SPIRou RVs for systematics. Gaussian processes (GP) modeling is applied to correct the SOPHIE RVs for stellar activity. The l1 and apodize sine periodogram (ASP) analysis is used to search for planetary signals in the SPIRou data taking into account activity indicators. We analyse TESS data and search for planetary transits. We report the detection of a M sin(i)=8.4+-1.3 Mearth sub-Neptune planet at a period of 6.020+-0.004 days in circular orbit with SPIRou. The same signal, although with lower significance, is also retrieved in the SOPHIE RV data after correction for activity using a GP trained on SPIRou's longitudinal magnetic field (Bl) measurements. TESS data indicates that the planet is not transiting. We find within the SPIRou wPCA RVs tentative evidence for two additional planetary signals at 2.99 and 18.7 days. In conclusion, infrared RVs are a powerful method to detect extrasolar planets around active M dwarfs, care should be taken however to correct/filter systematics generated by residuals of the telluric correction or small structures in the detector plane. The LBL technique combined with PCA offers a promising way to reach this objective. Further monitoring of Gl 410 is necessary.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.03331-b31b1b.svg)](https://arxiv.org/abs/2504.03331) | **The Galactic-Centre Arms inferred from ACES (ALMA CMZ Exploration Survey)**  |
|| Y. Sofue, et al. -- incl., <mark>P. Garcia</mark> |
|*Appeared on*| *2025-04-07*|
|*Comments*| *Accepted for PASJ, 20 pages, 19 figures*|
|**Abstract**|            Analyzing longitude-velocity diagrams (LVDs) in the CS(J=2-1) and H13CN(J=1-0) molecular lines from the internal release data of the ALMA Central-Molecular-Zone Exploration Survey (ACES) and in the 13CO (J=1-0) line from the Nobeyama Galactic-Centre (GC) survey, we identify six GC Arms as prominent straight LV ridges. In addition to the currently known Arms I to IV, we identify a new inner arm, Arm V, and further highlight the circum-nuclear disc (CND) as Arm VI. Integrated intensity maps of the Arms on the sky suggest that most of the Arms compose ring-like structures inclined from the Galactic plane. We determine the radii (curvatures) of the Arms using the velocity-gradient ($dv/dl$) method, assuming that the arms are rotating on circular orbits at a constant velocity of $\sim 150$ km/s. We show that Arms I and II compose the main ring structure of the CMZ with radii $\sim 100$--120 pc; Arm III is a dense arm 42 pc from the GC; Arm IV is a clear and narrow arm 20 pc from the GC; and Arm V is a faint, long arm of 8.2 pc radius. We show that the circum-nuclear disc (CND) composes the sixth arm, Arm VI, of radius $\sim 2.3$ pc associated with bifurcated spiral fins. We also discuss the association of the 20- and 50-km/s clouds with these Arms. The radii of the arms fall on an empirical relation $R\sim 630 (2/5)^N$ for $N=1$ (Arm I) to 6 (VI), suggesting either discrete rings or a logarithmic spiral with pitch angle $\sim 22^\circ$. The vertical full extent of the arm increases with radius and is represented by $z\sim 0.7 (R/1 {\rm pc})^{0.7}$ pc. The tilt angle of the arms from the Galactic plane, or the warping, increases rapidly toward the GC.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['', '', 'tmp_2504.02908/./confidence_interval_hees.png', 'tmp_2504.02908/./comparison_newton.png']
file not found 
file not found 
copying  tmp_2504.02908/./confidence_interval_hees.png to _build/html/
copying  tmp_2504.02908/./comparison_newton.png to _build/html/
exported in  _build/html/2504.02908.md
    + _build/html/
    + _build/html/
    + _build/html/tmp_2504.02908/./confidence_interval_hees.png
    + _build/html/tmp_2504.02908/./comparison_newton.png
found figures ['tmp_2504.03364/./Fig/sensmap_average.png', 'tmp_2504.03364/./Fig/Phased_RV_P1_1d_nores_nobin_errors_parlist.png', 'tmp_2504.03364/./Fig/GLS_signal_search_J01048-181.png']
copying  tmp_2504.03364/./Fig/sensmap_average.png to _build/html/
copying  tmp_2504.03364/./Fig/Phased_RV_P1_1d_nores_nobin_errors_parlist.png to _build/html/
copying  tmp_2504.03364/./Fig/GLS_signal_search_J01048-181.png to _build/html/
exported in  _build/html/2504.03364.md
    + _build/html/tmp_2504.03364/./Fig/sensmap_average.pn

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\La}{\mathcal{L}}$
$\newcommand{\Lb}{\pazocal{L}}$
$\newcommand{\af}[1]{{\textcolor{red}{\sf{[Arianna: #1]}} }}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\usepackage{calrsfs}$
$\DeclareMathAlphabet{\pazocal}{OMS}{zplm}{m}{n}$
$\newcommand{\La}{\mathcal{L}}$
$\newcommand{\Lb}{\pazocal{L}}$
$\newcommand{\af}[1]{{\textcolor{red}{\sf{[Arianna: #1]}} }}$
$\begin{document}$
$   \title{On the presence of a fifth force at the Galactic Center}$
$\author{$
$    The GRAVITY Collaboration\fnmsep\thanks{$
$    GRAVITY is developed in collaboration by MPE, LESIA of Paris Observatory / CNRS / Sorbonne Université / Univ. Paris Diderot, and IPAG of Université Grenoble Alpes / CNRS, MPIA, Univ. of Cologne, CENTRA - Centro de Astrofisica e Gravitação, and ESO. }$
$    :$
$    K.~Abd El Dayem             \inst{1}            \and$
$    R.~Abuter                   \inst{4}            \and$
$    N.~Aimar                    \inst{10, 7}            \and$
$    P.~Amaro Seoane             \inst{14,2,19}   \and$
$    A.~Amorim                   \inst{8,7}          \and$
$    J.P.~Berger                 \inst{3,4}          \and$
$    H.~Bonnet                   \inst{4}            \and$
$    G.~Bourdarot                \inst{2}            \and$
$    W.~Brandner                 \inst{5}            \and$
$    V.~Cardoso                  \inst{7,16}         \and$
$    Y.~Clénet                   \inst{1}            \and$
$    R.~Davies                   \inst{2}            \and$
$    P.T.~de~Zeeuw               \inst{20}           \and$
$    A.~Drescher                 \inst{2}            \and$
$    A.~Eckart                   \inst{6,13}         \and$
$    F.~Eisenhauer               \inst{2,18}         \and$
$    H.~Feuchtgruber             \inst{2}            \and$
$    G.~Finger                   \inst{2}            \and$
$    N.M.~Förster~Schreiber      \inst{2}            \and$
$    A.~Foschi                   \inst{1, 2}\thanks{Corresponding author: A.~Foschi (arianna.foschi{@}obspm.fr).}          \and$
$    P.~Garcia                   \inst{10,7}         \and$
$    E.~Gendron                  \inst{1}            \and$
$    R.~Genzel                   \inst{2,11}         \and$
$    S.~Gillessen                \inst{2}            \and$
$    M.~Hartl                    \inst{2}            \and$
$    X.~Haubois                  \inst{9}            \and$
$    F.~Haussmann                \inst{2}            \and$
$    T.~Henning                  \inst{5}            \and$
$    S.~Hippler                  \inst{5}            \and$
$    M.~Horrobin                 \inst{6}            \and$
$    L.~Jochum                   \inst{9}            \and$
$    L.~Jocou                    \inst{3}            \and$
$    A.~Kaufer                   \inst{9}            \and$
$    P.~Kervella                 \inst{1}            \and$
$    S.~Lacour                   \inst{1,4}            \and$
$    V.~Lapeyrère                \inst{1}            \and$
$    J.-B.~Le~Bouquin            \inst{3}            \and$
$    P.~Léna                     \inst{1}            \and$
$    D.~Lutz                     \inst{2}            \and$
$    F.~Mang                     \inst{2}            \and$
$    N.~More                     \inst{2}            \and$
$    J.~Osorno                   \inst{1}            \and$
$    T.~Ott                      \inst{2}            \and$
$    T.~Paumard                  \inst{1}            \and$
$    K.~Perraut                  \inst{3}            \and$
$    G.~Perrin                   \inst{1}            \and$
$    S.~Rabien                   \inst{2}            \and$
$    D.~C.~Ribeiro               \inst{2}            \and$
$    M.~Sadun Bordoni            \inst{2}            \and$
$    S.~Scheithauer              \inst{5}            \and$
$    J.~Shangguan                \inst{21}            \and$
$    T.~Shimizu                  \inst{2}            \and$
$    J.~Stadler                  \inst{12,2}         \and$
$    O.~Straub                   \inst{2,17}         \and$
$    C.~Straubmeier              \inst{6}            \and$
$    E.~Sturm                    \inst{2}            \and$
$    L.J.~Tacconi                \inst{2}            \and$
$    I.~Urso                     \inst{1}            \and$
$    F.~Vincent                  \inst{1}            \and$
$    S.D.~von~Fellenberg           \inst{13,2}         \and$
$    E.~Wieprecht                \inst{2}            \and$
$    J.~Woillez                  \inst{4}$
$}$
$    \institute{$
$            LIRA, Observatoire de Paris, Université PSL, CNRS, Sorbonne Université, Université de Paris, 5 place Jules Janssen, 92195 Meudon, France \and$
$  	    Max Planck Institute for Extraterrestrial Physics, Giessenbachstraße 1, 85748 Garching, Germany \and$
$  	    Univ. Grenoble Alpes, CNRS, IPAG, 38000 Grenoble, France \and$
$  	    European Southern Observatory, Karl-Schwarzschild-Straße 2, 85748 Garching, Germany \and$
$  	    Max Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany \and$
$  	    1st Institute of Physics, University of Cologne, Zülpicher Straße 77, 50937 Cologne, Germany \and$
$  	    CENTRA - Centro de Astrofísica e Gravitação, IST, Universidade de Lisboa, 1049-001 Lisboa, Portugal \and$
$  	    Universidade de Lisboa - Faculdade de Ciências, Campo Grande, 1749-016 Lisboa, Portugal \and$
$  	    European Southern Observatory, Casilla 19001, Santiago 19, Chile \and$
$  	    Faculdade de Engenharia, Universidade do Porto, rua Dr. Roberto Frias, 4200-465 Porto, Portugal \and$
$  	    Departments of Physics \& Astronomy, Le Conte Hall, University of California, Berkeley, CA 94720, USA \and$
$  	    Max Planck Institute for Astrophysics, Karl-Schwarzschild-Straße 1, 85748 Garching, Germany \and$
$  	    Max Planck Institute for Radio Astronomy, auf dem Hügel 69, 53121 Bonn, Germany \and$
$  	    Institute of Multidisciplinary Mathematics, Universitat Politècnica de València, València, Spain \and$
$  	    Advanced Concepts Team, ESA, TEC-SF, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands \and$
$  	    Center of Gravity, Niels Bohr Institute, Blegdamsvej 17, 2100 Copenhagen, Denmark \and$
$  	    ORIGINS Excellence Cluster, Boltzmannstraße 2, 85748 Garching, Germany \and$
$  	    Department of Physics, Technical University of Munich, 85748 Garching, Germany \and$
$  	    Higgs Centre for Theoretical Physics, Edinburgh, UK \and$
$            Leiden University, 2311 EZ Leiden, The Netherlands \and$
$            The Kavli Institute for Astronomy and Astrophysics, Peking University, Beijing 100871, China$
$    }$
$   \date{$
$   }$
$  \abstract$
$    $
$   {The presence of a Yukawa-like correction to Newtonian gravity is investigated at the Galactic Center, leading to a new upper limit for the intensity of such a correction.}$
$   {We perform a Markov Chain Monte Carlo analysis using the astrometric and spectroscopic data of star S2 collected at the Very Large Telescope by GRAVITY, NACO and SINFONI instruments, covering the period from 1992 to 2022.}$
$   {The precision of the GRAVITY instrument allows us to derive the most stringent upper limit at the Galactic Center for the intensity of the Yukawa contribution (\propto   \alpha e^{- \lambda r}) to be |\alpha| < 0.003 for a scale length \lambda = 3 \cdot 10^{13}  \rm m   (\sim 200   \rm  AU). This improves by roughly one order of magnitude all estimates obtained in previous works. }$
$    $
$   \keywords{black holes physics --$
$                Galaxy:centre --$
$                gravitation$
$               }$
$   \maketitle$
$\n\end{document}\end{equation}}$
$\newcommand{\eeq}{\end{equation}}$</div>



<div id="title">

# On the presence of a fifth force at the Galactic Center

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.02908-b31b1b.svg)](https://arxiv.org/abs/2504.02908)<mark>Appeared on: 2025-04-07</mark> -  _Submitted to A&A_

</div>
<div id="authors">

T. G. Collaboration, et al. -- incl., <mark>W. Brandner</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>S. Scheithauer</mark>

</div>
<div id="abstract">

**Abstract:**            Aims: The presence of a Yukawa-like correction to Newtonian gravity is investigated at the Galactic Center, leading to a new upper limit for the intensity of such a correction. Methods: We perform a Markov Chain Monte Carlo analysis using the astrometric and spectroscopic data of star S$2$ collected at the Very Large Telescope by GRAVITY, NACO and SINFONI instruments, covering the period from $1992$ to $2022$. Results: The precision of the GRAVITY instrument allows us to derive the most stringent upper limit at the Galactic Center for the intensity of the Yukawa contribution ($\propto \, \alpha e^{- \lambda r}$) to be $|\alpha| < 0.003$ for a scale length $\lambda = 3 \cdot 10^{13}\, \rm m\, (\sim 200 \, \rm AU)$. This improves by roughly one order of magnitude all estimates obtained in previous works.         

</div>

<div id="div_fig1">

<img src="" alt="Fig5.1" width="50%"/><img src="" alt="Fig5.2" width="50%"/>

**Figure 5. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued. (*FigVibStab*)

</div>
<div id="div_fig2">

<img src="tmp_2504.02908/./confidence_interval_hees.png" alt="Fig7" width="100%"/>

**Figure 7. -** $95\%$ confidence level on $|\alpha|$ obtained in this work (red dots) compared with previous estimates by \protectHees:2017aal(blue dots). (*fig:uncertainties_alpha*)

</div>
<div id="div_fig3">

<img src="tmp_2504.02908/./comparison_newton.png" alt="Fig6" width="100%"/>

**Figure 6. -** Comparison of the posterior distributions $P(|\alpha||D)$ between the Keplerian model (black curve) and  the $1$PN model (red curve).
 (*fig:comparison_newton*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.02908"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\AS}[3]{#1^{+#2}_{-#3}}$
$\newcommand{\Commref}[1]{\textcolor{red}{\textbf{#1}}}$
$\newcommand{\TODO}[1]{\textcolor{magenta}{\textsc{todo:} \textit{#1}}}$
$\newcommand{\StarJOne}{G~261--6}$
$\newcommand{\PlanetJOne}{G~261--6 b}$
$\newcommand{\MassJOne}{\SI{1.37\pm0.23}{M_\oplus}}$
$\newcommand{\PeriodJOne}{\SI{5.4536\pm0.0032}{\day}}$
$\newcommand{\ProtJOne}{\SI{114}{\day}}$
$\newcommand{\PeriodCandJone}{\SI{1.83}{\day}}$
$\newcommand{\StarJTwo}{G~192--15}$
$\newcommand{\MassJTwoc}{\SI{14.3}{M_\oplus}}$
$\newcommand{\MassJTwob}{\SI{1.03\pm0.18}{M_\oplus}}$
$\newcommand{\StarJZero}{G~268--110}$
$\newcommand{\PlanetJZero}{G~268--110 b}$
$\newcommand{\MassJZero}{\SI{1.52\pm0.25}{M_\oplus}}$
$\newcommand{\PeriodJZero}{\SI{1.43263\pm0.000076}{\day}}$
$\newcommand{\TeqJZero}{\SI{534\pm12}{\day}}$
$\newcommand{\ProtJZero}{\SI{143}{\day}}$
$\newcommand{\RomanNumeralCaps}[1]$
$\newcommand{\inst}[1]{\unskip^{\instrefs{#1}}}$
$\newcommand{\autoref}$
$\newcommand{\scsep}{\newcommand\scsep{,}}$
$\newcommand{\equationautorefname}{Eq.}$
$\newcommand{\figureautorefname}{Fig.}$
$\newcommand{\sectionautorefname}{Sect.}$
$\newcommand{\subsectionautorefname}{Sect.}$
$\newcommand{\subsubsectionautorefname}{Sect.}$
$\newcommand\scsep{,}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs: Occurrence rates of Earth-like planets around very low-mass stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.03364-b31b1b.svg)](https://arxiv.org/abs/2504.03364)<mark>Appeared on: 2025-04-07</mark> -  _Number of pages: 25; number of figures: 23 (12 in main + 11 in appendix). Accepted in A&A; DOI (pending): https://doi.org/10.1051/0004-6361/202453381 (Volume 696, Article number A101)_

</div>
<div id="authors">

A. Kaminski, et al. -- incl., <mark>R. Burn</mark>, <mark>T. Henning</mark>, <mark>M. Kürster</mark>

</div>
<div id="abstract">

**Abstract:**            Aims: Previous estimates of planet occurrence rates in the CARMENES survey indicated increased numbers of planets on short orbits for M dwarfs with masses below 0.34\,M$_\odot$. Here we focused on the lowest-mass stars in the survey, comprising 15 inactive targets with masses under 0.16\,M$_\odot$. Methods: To correct for detection biases, we determined detection sensitivity maps for individual targets and the entire sample. Using Monte Carlo simulations, we estimated planet occurrence rates for orbital periods of 1\,d to 100\,d and minimum masses from 0.5\,M$_\oplus$ to 10\,M$_\oplus$. Results: The radial velocity (RV) data from CARMENES reveal four new planets around three stars in our sample, namely G~268--110\,b, G~261--6\,b, and G~192--15\,b and c. All three b planets have minimum masses of 1.03--1.52\,M$_\oplus$ and orbital periods of 1.43--5.45\,d, while G~192--15\,c is a 14.3\,M$_\oplus$ planet on a wide, eccentric orbit with $P \approx 1218$\,d and $e \approx 0.68$. Our occurrence rates suggest considerable dependencies with respect to stellar masses. For planets below 3\,M$_\oplus$ we found rates consistent with one planet per star across all investigated periods, but the rates decrease almost by an order of magnitude for larger planet masses up to 10\,M$_\oplus$. Compared to previous studies, low-mass stars tend to harbor more planets with $P <10$\,d. We also demonstrate that synthetic planet populations based on the standard core accretion scenario predict slightly more massive planets on wider orbits than observed. Conclusions: Our findings confirm that planet occurrence rates vary with stellar masses even among M dwarfs, as we found more planets with lower masses and on shorter orbits in our subsample of very low-mass stars compared to more massive M dwarfs. Therefore, we emphasize the need for additional differentiation in future studies.         

</div>

<div id="div_fig1">

<img src="tmp_2504.03364/./Fig/sensmap_average.png" alt="Fig3" width="100%"/>

**Figure 3. -** CARMENES detection sensitivity map, averaged over the individual maps of the 15 stellar targets of this study.
    The light blue markers indicate the 11 planets included for this occurrence rate analysis, and the color map illustrates the detection probabilities of the respective period-mass grid points. The solid lines represent the masses associated to the RV semi-amplitude values equal to the RVs' root mean square averaged over the time series (blue), the mean RV uncertainties (magenta), and the median of the RV uncertainties. (*fig:planet_occurrence*)

</div>
<div id="div_fig2">

<img src="tmp_2504.03364/./Fig/Phased_RV_P1_1d_nores_nobin_errors_parlist.png" alt="Fig1" width="100%"/>

**Figure 1. -** Phased RV plot for $\PlanetJZero$  based on the best fit model (1P$_\text{(1.43 d-circ)}$). The black line depicts the model based on the parameters listed in $\autoref${tab:planetparams}. The shaded area illustrates the 1$\sigma$ confidence interval. (*fig:phasefolded_J01048-181*)

</div>
<div id="div_fig3">

<img src="tmp_2504.03364/./Fig/GLS_signal_search_J01048-181.png" alt="Fig6" width="100%"/>

**Figure 6. -** Window function (upper panel) and GLS periodograms of the CARMENES RVs for $\StarJZero$  before ('0P', middle) and after subtracting the one-planet model ('1P', bottom).
    The two panels on the left and right represent the same GLS periodograms but plot different regions to better represent the occurring signals. The period of the 1.43-day planet is highlighted by a red solid line. Its first-order aliases at \SI{3.28}{\day} and \SI{0.59}{\day} are marked by red dashed lines, and the second-order aliases at \SI{0.76}{\day} and \SI{0.43}{\day} by red dotted lines. The rotation period of \SI{143}{\day} determined by \cite{Newton2018} is indicated by a solid green line. (*fig:gls_rv_J01048-181*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.03364"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vsys}{v_{\rm sys}}$
$\newcommand\MJ{M_{\mathrm{Jup}}}$
$\newcommand\RJ{R_{\mathrm{Jup}}}$
$\newcommand\RS{R_{\odot}}$
$\newcommand\MS{M\mathrm{_\odot}}$
$\newcommand\LS{L_{\odot}}$
$\newcommand\teff{T_\mathrm{eff}}$
$\newcommand\kms{km s^{-1}}$
$\newcommand\ms{m s^{-1}}$
$\newcommand{\1}{s}$
$\newcommand\t{0}$
$\newcommand\sls{\smallcapital SLS}$
$\newcommand\mp{M_{\rm p}}$
$\newcommand\rt{\mathrm{R_{\oplus}}}$
$\newcommand\mt{\mathrm{M_{\oplus}}}$
$\newcommand\starmass{M_\mathrm{s}}$
$\newcommand\gl{Gl~410}$
$\newcommand\spirou{SPIRou}$</div>



<div id="title">

# Characterizing planetary systems with SPIRou: Detection of a sub-Neptune in a 6 day period orbit   around the M dwarf Gl 410$\thanks{Based on observations obtained with the spectropolarimeter SPIRou at the Canada-France-Hawaii Telescope (CFHT).	  }$          

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.03572-b31b1b.svg)](https://arxiv.org/abs/2504.03572)<mark>Appeared on: 2025-04-07</mark> -  _Accepted by A&A, 4 April 2025; 35 pages_

</div>
<div id="authors">

A. Carmona, et al. -- incl., <mark>L. Acuña</mark>

</div>
<div id="abstract">

**Abstract:** The search of exoplanets around nearby M dwarfs is a crucial milestone to perform     the census of planetary systems in the vicinity of our Solar System. Since 2018 our team is carrying a blind search program for planets around nearby M dwarfs   with the near-IR spectro-polarimeter and velocimeter SPIRou at the CFHT and the optical velocimeter SOPHIE at the OHP in France.   The aim of this paper is to present our results on Gl 410, a 0.55 M $_\odot$ 480 $\pm$ 150 Myr old active M dwarf distant 12 pc. We search for planetary companions using radial velocities (RVs).   We use the line-by-line (LBL) technique to measure the RVs with SPIRou and the template matching method with SOPHIE.   Three different methods, two based in principal component analysis (PCA),   are used to clean the SPIRou RVs for systematics.   Gaussian processes (GP) modeling is applied to correct the SOPHIE RVs for stellar activity.   The $\ell_1$ and apodize sine periodogram (ASP) analysis is used to search for planetary signals in the SPIRou data taking into account activity indicators.   We analyse TESS data and search for planetary transits. We report the detection of a $M$ sin $(i)$ =8.4 $\pm$ 1.3 M $_\oplus$ sub-Neptune planet at a period of 6.020 $\pm$ 0.004 days   in circular orbit with SPIRou.   The same signal, although with lower significance, is also retrieved in the SOPHIE RV data after correction for activity   using a GP trained on SPIRou's longitudinal magnetic field ( $B_\ell$ ) measurements.   TESS data indicates that the planet is not transiting.   We find within the SPIRou wPCA RVs tentative evidence for two additional planetary signals at 2.99 and 18.7 days. Infrared RVs are a powerful method to detect extrasolar planets around active M dwarfs. Care should be taken however to correct/filter   systematics generated by residuals of the telluric correction or small structures in the detector plane.   The LBL technique combined with PCA offers a promising way to reach this objective.   Further monitoring of Gl 410 is necessary.

</div>

<div id="div_fig1">

<img src="tmp_2504.03572/./GL410_lbl2bervzp.png" alt="Fig31.1" width="33%"/><img src="tmp_2504.03572/./GL410_wPCA.png" alt="Fig31.2" width="33%"/><img src="tmp_2504.03572/./GL410_Wapiti.png" alt="Fig31.3" width="33%"/>

**Figure 31. -** SPIRou RV measurements as a function of time and { GLS} periodograms for the "raw" LBL measurements
and the PCA corrected LBL measurements obtained with wPCA (Artigau et al., in prep) and Wapiti \citep[][]{Wapiti2023}.
Blue dots indicate the measurements taken at $|V_{\rm tot}|>10$ km s$^{-1}$.
Red dots are the measurements obtained when $|V_{\rm tot}|<10$ km s$^{-1}$(i.e. moment of the highest influence of the atmosphere on the
stellar spectrum).
In the periodograms,
the grey horizontal lines indicate the { bootstrap-calculated} 10\%, 1\% and 0.1\% FAP levels.
A summary of the statistics of the each of the time-series is provided in Table \ref{table:SPIRou_RV_periodogram}. (*fig:SPIRou_RV_periodogram*)

</div>
<div id="div_fig2">

<img src="tmp_2504.03572/./GL410_wPCA_2planet.png" alt="Fig8" width="100%"/>

**Figure 8. -** Periodograms of the SPIRou wPCA data
    after subtraction of the circular orbit models of Gl 410b and the candidate planetary signal at 18.7 days.
     (*GL410_wPCA_2planet*)

</div>
<div id="div_fig3">

<img src="tmp_2504.03572/./GL410_corner_bell.png" alt="Fig19" width="100%"/>

**Figure 19. -** Corner plot of the quasi-periodic GP fit of the SPIRou longitudinal magnetic field (B$_\ell$) time-series of  Gl 410.
    We provide the description of  the meaning of the variables in Table \ref{GP_table}. (*fig:corner_bell*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.03572"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

429  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

17  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
